# Early Stopping or Smaller Ensembles

*This notebook first appeared as a [blog post](//betatim.github.io/posts/bumping) on [Tim Head](//betatim.github.io)'s blog.*

*License: [MIT](http://opensource.org/licenses/MIT)*

*(C) 2015, Tim Head.*
*Feel free to use, distribute, and modify with the above attribution.*

When building an ensemble of trees (like in a Random Forest or gradient boosting) one question keeps coming up: how many base learners should I add to my ensemble?

This post shows you how to keep growing your ensemble until the test error reaches a minimum. This means you do not end up wasting time waiting for your ensemble to build 1000 trees if you only need 200.

First a few imports and dataset fetching to get us started.

In [1]:
%matplotlib inline

In [2]:
%%bash
wget --quiet -O /tmp/HIGGS_background.root http://files.figshare.com/1920208/HIGGS_background.root
wget --quiet -O /tmp/HIGGS_signal.root http://files.figshare.com/1920200/HIGGS_signal.root

In [ ]:
import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns

from root_numpy import root2array, rec2array

from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier

## Loading ROOT Trees

Data comes first, the excellent [`root_numpy`](//rootpy.github.io/root_numpy/) library makes it easy to read your data stored in a [ROOT](http://root.cern.ch) TTree. Each call to `root2array` will create a 2D array which contains one row per event, and one column representing each branch you want to use.

You can download the dataset used in this post from [figshare](//figshare.com):
 * [HIGGS-background.root](http://figshare.com/articles/HIGGS_background/1314900)
 * [HIGGS-signal.root](http://figshare.com/articles/HIGGS/1314899)
 
The events are derived from the much larger [HIGGS](http://archive.ics.uci.edu/ml/datasets/HIGGS) dataset. A description of the variables can be found there as well.

In [ ]:
# the first branch tells you the class of each sample, which is redundant here
signal = root2array("/tmp/HIGGS_signal.root")
signal = rec2array(signal)[:,1:]

backgr = root2array("/tmp/HIGGS_background.root")
backgr = rec2array(backgr)[:,1:]

# Organise data into one 2D array of shape (n_samples x n_features)
X = np.concatenate((signal, backgr))
y = np.concatenate((np.ones(signal.shape[0]), np.zeros(backgr.shape[0])))

## The big split

In addition to loading the data we directly split it into a development and evaluation set. This is good practice in general, here we use it to artificially shrink the dataset and have "fresh" copies to evaluate classifier performance on.

The development set gets split further into a training and testing set.

In [ ]:
# set aside an evaluation sample
# We artificially limit ourselves
# to a small subset of the available
# samples. This speeds things up and
# makes it easier to illustrate things
# like over-fitting. In real life you
# would not do this.
X_dev,X_eval, y_dev,y_eval = train_test_split(X, y,
                                              train_size=20000,
                                              test_size=10000,
                                              random_state=42)

# Split development set into a train and test sample
X_train,X_test, y_train,y_test = train_test_split(X_dev, y_dev,
                                                  test_size=0.33, random_state=4685)

In [ ]:
clf = GradientBoostingClassifier(max_depth=4,
                                 learning_rate=0.2,
                                 n_estimators=200)
_ = clf.fit(X_train, y_train)

In [ ]:
def validation_curve(clf):
    test_score = np.empty(len(clf.estimators_))
    train_score = np.empty(len(clf.estimators_))

    for i, pred in enumerate(clf.staged_predict_proba(X_test)):
        test_score[i] = 1-roc_auc_score(y_test, pred[:,1])

    for i, pred in enumerate(clf.staged_predict_proba(X_train)):
        train_score[i] = 1-roc_auc_score(y_train, pred[:,1])

    best_iter = np.argmin(test_score)
    test_line = plt.plot(test_score)

    colour = test_line[-1].get_color()
    plt.plot(train_score, '--', color=colour)

    plt.xlabel("Number of boosting iterations")
    plt.ylabel("1 - area under ROC")
    plt.axvline(x=best_iter, color=colour)
    
validation_curve(clf)

In [ ]:
from sklearn.base import ClassifierMixin, clone
from functools import partial


def one_minus_roc(X, y, est):
    pred = est.predict_proba(X)[:, 1]
    return 1-roc_auc_score(y, pred)


class EarlyStopping(ClassifierMixin):
    def __init__(self, estimator, max_n_estimators, scorer, scale=1.01):
        self.estimator = estimator
        self.max_n_estimators = max_n_estimators
        self.scorer = scorer
        self.scale = scale
    
    def _make_estimator(self, append=True):
        """Make and configure a copy of the `estimator` attribute."""
        estimator = clone(self.estimator)
        estimator.n_estimators = 1
        estimator.warm_start = True
        return estimator
    
    def fit(self, X, y):
        est = self._make_estimator()
        self.scores_ = []

        for n_est in xrange(1, self.max_n_estimators+1):
            est.n_estimators = n_est
            est.fit(X,y)
            
            score = self.scorer(est)
            self.estimator_ = est
            self.scores_.append(score)

            if score > self.scale*np.min(self.scores_):
                return self

        return self

In [ ]:
def stop_early(classifier, scale=1.01):
    early = EarlyStopping(classifier,
                          max_n_estimators=200,
                          scale=scale,
                          scorer=partial(one_minus_roc, X_test, y_test))
    early.fit(X_train, y_train)
    plt.plot(np.arange(1, 200)[:len(early.scores_)],
             early.scores_)
    plt.xlabel("number of estimators")
    plt.ylabel("1 - area under ROC")
    
stop_early(GradientBoostingClassifier(learning_rate=0.2, max_depth=4))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

stop_early(RandomForestClassifier(max_depth=4))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

stop_early(BaggingClassifier(DecisionTreeClassifier(max_depth=4)), scale=1.03)